# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [4]:
# Your code here
scaler = StandardScaler()

# Scale X_train and X_val using StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X.columns)
# Ensure X_train and X_val are scaled DataFrames
# (hint: you can set the columns using X.columns)

In [5]:
# Your code here
model = LinearRegression()
# Create a LinearRegression model and fit it on scaled training data
model.fit(X_train_scaled, y_train)

# Calculate a baseline r-squared score on training data
r2_train = model.score(X_train_scaled, y_train)
r2_train

0.7868344817421309

## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [6]:
# Your code here

# Set up data structure
interaction_results = []

# Find combinations of columns and loop over them
# Make copies of X_train and X_val

for combo in combinations(X_train.columns, 2):
    X_train_copy = X_train_scaled.copy()
    X_val_copy = X_val_scaled.copy()
    
    # Add interaction term to data
    X_train_copy[f"{combo[0]}_x_{combo[1]}"] = X_train_copy[combo[0]] * X_train_copy[combo[1]]
    X_val_copy[f"{combo[0]}_x_{combo[1]}"] = X_val_copy[combo[0]] * X_val_copy[combo[1]]
    
    # Find r-squared score (fit on training data, evaluate on validation data)
    model.fit(X_train_copy, y_train)
    r2_val = model.score(X_val_copy, y_val)
    
    # Append to data structure
    interaction_results.append((combo, r2_val))
    
    
# Sort and subset the data structure to find the top 7
interaction_results_sorted = sorted(interaction_results, key=lambda x: x[1], reverse=True)[:7]
for interaction, r2 in interaction_results_sorted:
    print(f"Interaction: {interaction}, R-squared: {r2}")

Interaction: ('LotArea', '1stFlrSF'), R-squared: 0.7211105666140568
Interaction: ('LotArea', 'TotalBsmtSF'), R-squared: 0.7071649207050109
Interaction: ('LotArea', 'GrLivArea'), R-squared: 0.6690980823779027
Interaction: ('LotArea', 'Fireplaces'), R-squared: 0.6529699515652588
Interaction: ('2ndFlrSF', 'TotRmsAbvGrd'), R-squared: 0.6472994890405193
Interaction: ('OverallCond', 'TotalBsmtSF'), R-squared: 0.642901987923377
Interaction: ('OverallQual', '2ndFlrSF'), R-squared: 0.6422324294284367


### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_val` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [7]:
# Your code here

# Loop over top 7 interactions
for interaction, _ in interaction_results_sorted:
    
    # Extract column names from data structure
    col1, col2 = interaction

    # Construct new column name
    new_col_name = f"{col1}_{col2}"

    
    # Add new column to X_train and X_val
X_train_scaled[new_col_name] = X_train_scaled[col1] * X_train_scaled[col2]
X_val_scaled[new_col_name] = X_val_scaled[col1] * X_val_scaled[col2]

## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [9]:
# Your code here

# Set up data structure
polynomial_results = []

# Loop over all columns
for col in X_train.columns:

    # Loop over degrees 2, 3, 4
     for degree in [2, 3, 4]:

        # Make a copy of X_train and X_val
        X_train_copy = X_train_scaled.copy()
        X_val_copy = X_val_scaled.copy()
        # Instantiate PolynomialFeatures with relevant degree
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
        X_train_poly = poly.fit_transform(X_train_copy[[col]])
        X_val_poly = poly.transform(X_val_copy[[col]])
        poly_col_names = [f"{col}_poly_{d}" for d in range(1, degree+1)]
        X_train_poly_df = pd.DataFrame(X_train_poly, columns=poly_col_names)
        X_val_poly_df = pd.DataFrame(X_val_poly, columns=poly_col_names)
        
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
        X_train_copy = X_train_copy.drop(columns=[col])
        X_val_copy = X_val_copy.drop(columns=[col])
        X_train_copy = pd.concat([X_train_copy, X_train_poly_df], axis=1)
        X_val_copy = pd.concat([X_val_copy, X_val_poly_df], axis=1)
    
        # Find r-squared score on validation
        model.fit(X_train_copy, y_train)
        r2_val = model.score(X_val_copy, y_val)
    
        # Append to data structure
        polynomial_results.append((col, degree, r2_val))

# Sort and subset the data structure to find the top 7
polynomial_results_sorted = sorted(polynomial_results, key=lambda x: x[2], reverse=True)[:7]
for col, degree, r2 in polynomial_results_sorted:
    print(f"Column: {col}, Degree: {degree}, R-squared: {r2}")


Column: GrLivArea, Degree: 3, R-squared: 0.8208554979315524
Column: GrLivArea, Degree: 4, R-squared: 0.7804048794615333
Column: 2ndFlrSF, Degree: 3, R-squared: 0.6759499371294796
Column: 2ndFlrSF, Degree: 4, R-squared: 0.6750327672736952
Column: 1stFlrSF, Degree: 2, R-squared: 0.6739854014775069
Column: 2ndFlrSF, Degree: 2, R-squared: 0.671827545846226
Column: OverallQual, Degree: 4, R-squared: 0.654817644533801


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [10]:
# Your code here
best_polynomials = {}

# Filter out duplicates
for col, degree, r2 in polynomial_results_sorted:
    if col not in best_polynomials or best_polynomials[col][1] < degree:
        best_polynomials[col] = (r2, degree)

# Loop over remaining results
for col, (r2, degree) in best_polynomials.items():
    X_train_copy = X_train_scaled.copy()
    X_val_copy = X_val_scaled.copy()

    # Create polynomial terms
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_copy[[col]])
    X_val_poly = poly.transform(X_val_copy[[col]])
    
    # Concat new polynomials to X_train and X_val
    X_train_copy = pd.concat([X_train_copy, X_train_poly_df], axis=1)
    X_val_copy = pd.concat([X_val_copy, X_val_poly_df], axis=1)
    

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [11]:
# Your code here
print("Training set columns after adding interactions and polynomials:")
print(X_train_copy.columns)

print("Validation set columns after adding interactions and polynomials:")
print(X_val_copy.columns)


Training set columns after adding interactions and polynomials:
Index(['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
       'OverallQual_2ndFlrSF', 'Fireplaces_poly_1', 'Fireplaces_poly_2',
       'Fireplaces_poly_3', 'Fireplaces_poly_4'],
      dtype='object')
Validation set columns after adding interactions and polynomials:
Index(['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd', 'GarageArea', 'Fireplaces',
       'OverallQual_2ndFlrSF', 'Fireplaces_poly_1', 'Fireplaces_poly_2',
       'Fireplaces_poly_3', 'Fireplaces_poly_4'],
      dtype='object')


## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and validation data.

In [12]:
# Your code here
# Initialize the linear regression model
model = LinearRegression()

# Fit the model on the training data (with interactions and polynomials)
model.fit(X_train_copy, y_train)

# Calculate R^2 score on the training data
train_r2 = model.score(X_train_copy, y_train)

# Calculate R^2 score on the validation data
val_r2 = model.score(X_val_copy, y_val)

# Print the R^2 scores for both training and validation sets
print(f"Training R^2: {train_r2:.4f}")
print(f"Validation R^2: {val_r2:.4f}")

Training R^2: 0.7945
Validation R^2: 0.6472


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and validation $R^2$ score and how many features remain.

In [16]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Initialize Linear Regression model
model = LinearRegression()

# Values for n_features_to_select to test
n_features_values = [5, 10, 15, 20, 25]

# Loop through each value of n_features_to_select
for n in n_features_values:
    # Initialize RFE with model and the number of features to select
    selector = RFE(model, n_features_to_select=n)
    
    # Fit RFE on training data
    selector.fit(X_train, y_train)
    
    # Get the selected features (this is a boolean mask)
    selected_features = selector.support_
    
    # Subset X_train and X_val using the selected features
    X_train_rfe = X_train.loc[:, selected_features]
    X_val_rfe = X_val.loc[:, selected_features]
    
    # Fit the model with selected features
    model.fit(X_train_rfe, y_train)
    
    # Calculate R-squared score on training data
    train_r2 = model.score(X_train_rfe, y_train)
    
    # Calculate R-squared score on validation data
    val_r2 = model.score(X_val_rfe, y_val)
    
    # Print results
    print(f"n_features_to_select = {n}")
    print(f"Training R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print(f"Features selected: {sum(selected_features)}\n")


n_features_to_select = 5
Training R^2: 0.7281
Validation R^2: 0.6817
Features selected: 5

n_features_to_select = 10
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10

n_features_to_select = 15
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10

n_features_to_select = 20
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10

n_features_to_select = 25
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10



Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [17]:
# Your code here
from sklearn.linear_model import Lasso

# Values for alpha to test
alpha_values = [0.1, 1, 10, 100]

# Loop through each value of alpha
for alpha in alpha_values:
    # Initialize Lasso with the current alpha
    lasso = Lasso(alpha=alpha)
    
    # Fit the Lasso model on the training data
    lasso.fit(X_train, y_train)
    
    # Get the selected features (those with non-zero coefficients)
    selected_features = np.sum(lasso.coef_ != 0)
    
    # Calculate R-squared score on training data
    train_r2 = lasso.score(X_train, y_train)
    
    # Calculate R-squared score on validation data
    val_r2 = lasso.score(X_val, y_val)
    
    # Print results
    print(f"alpha = {alpha}")
    print(f"Training R^2: {train_r2:.4f}")
    print(f"Validation R^2: {val_r2:.4f}")
    print(f"Features selected: {selected_features}\n")



alpha = 0.1
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10

alpha = 1
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10

alpha = 10
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10

alpha = 100
Training R^2: 0.7868
Validation R^2: 0.6376
Features selected: 10



Compare the results. Which features would you choose to use?

In [18]:
# Your written answer here
"""
In conclusion, I will choose the feature selection method that provides the best balance of model performance (based on validation 𝑅2) and the number of features. 
I can also select the model with the highest validation and the least number of features for a simpler, more interpretable model.
"""

'\nIn conclusion, I will choose the feature selection method that provides the best balance of model performance (based on validation 𝑅2) and the number of features. \nI can also select the model with the highest validation and the least number of features for a simpler, more interpretable model.\n'

## Evaluate Final Model on Test Data

### Data Preparation

At the start of this lab, we created `X_test` and `y_test`. Prepare `X_test` the same way that `X_train` and `X_val` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [23]:
# Your code here
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

# Loop over the top 7 interactions
top_interactions = [
    GrLivArea, 
    GrLivArea,
    2ndFlrSF,
    2ndFlrSF, 
    1stFlrSF, 
    2ndFlrSF,
    OverallQual, 
]

for col1, col2 in top_interactions:  
    X_test_scaled[f'{col1}_{col2}'] = X_test_scaled[col1] * X_test_scaled[col2]

# Loop over the top 7 polynomials
for col_name, degree, _ in top_polynomials:  
    poly = PolynomialFeatures(degree)
    
    col_data = X_test_scaled[[col_name]]
    
    # Transform the column into polynomial features
    poly_features = poly.fit_transform(col_data)
    
    poly_col_names = [f"{col_name}_poly_{i}" for i in range(1, poly_features.shape[1])]
    poly_features = poly_features[:, 1:]  
    
    # Create DataFrame for polynomial terms
    poly_df = pd.DataFrame(poly_features, columns=poly_col_names)
    
    # Concatenate the new polynomial terms to X_test_scaled
    X_test_scaled = pd.concat([X_test_scaled, poly_df], axis=1)



SyntaxError: invalid syntax (<ipython-input-23-fa47d1ea184f>, line 12)

### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + validation set, then find R-Squared and MSE values for the test set.

In [28]:
# Your code here
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Fit Lasso on the combined train + validation data
X_train_val = pd.concat([X_train, X_val])  # Combine train and validation sets
y_train_val = pd.concat([y_train, y_val])

# Initialize the Lasso model
lasso = Lasso(alpha=0.1)  # You can adjust alpha as needed

# Fit the Lasso model on the combined train + validation set
lasso.fit(X_train_val, y_train_val)

# Select features from Lasso (non-zero coefficients)
X_train_val_selected = X_train_val.loc[:, lasso.coef_ != 0]

# Train the model on the selected features
lasso.fit(X_train_val_selected, y_train_val)

# Evaluate the model on the test set
X_test_selected = X_test.loc[:, lasso.coef_ != 0]  
y_pred = lasso.predict(X_test_selected)

# Calculate R-squared and MSE
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"R-Squared: {r2:.4f}")
print(f"Mean Squared Error: {mse:.4f}")


R-Squared: 0.8020
Mean Squared Error: 1386804766.2206


## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.